In [1]:
import pandas as pd
import numpy
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
#%matplotlib inline
#plt.rcParams['figure.figsize'] = (16, 9)
#plt.style.use('ggplot')
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import scipy
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from collections import Counter
import joblib
from scipy.integrate import simps
from datetime import datetime
import sklearn.metrics as metrics
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from keras.layers import Dropout
from keras.regularizers import l2
from keras import initializers
from keras.models import model_from_json

#from xgboost import XGBClassifier

def norm_minimax(datos_train, datos_test=None, splits=True, mini=None, maxi=None):
    if (mini is None) and (maxi is None):
        maxi=(np.max(datos_train))
        mini=(np.min(datos_train))
        datos_train = (datos_train-mini)/(maxi-mini)
        return datos_train, mini, maxi
    if (mini is not None) & (maxi is not None) & (splits==True):
        datos_train = (datos_train-mini)/(maxi-mini)
        datos_test = (datos_test-mini)/(maxi-mini)
        return datos_train, datos_test
    if (mini is not None) & (maxi is not None) & (splits==False):
        datos_train = (datos_train-mini)/(maxi-mini)
        return datos_train
    print('Error de entradas en nomalizacion minimax')
    return -1

# calcular peso de clase para darselo al modelo
def pesos_clases(y_train, only_pos=False):
    cant_refa = 0
    cant_norefa = 0
    for i in range(0, len(y_train)):
        if y_train[i] == 1:
            cant_refa = cant_refa + 1
        else:
            cant_norefa = cant_norefa + 1
    total = cant_norefa + cant_refa
    porcentaje_refa = (cant_refa/total)*100
    print('Porcentaje de refacturas: %0.2f' % porcentaje_refa)
    weight_for_0 = (1 / cant_norefa)*(total)/2.0 
    weight_for_1 = ((1 / cant_refa)*(total)/2.0) #/2 #dividido 2 para que no haya tantos falsos positivos
    class_weight = {0: weight_for_0, 1: weight_for_1}
    initial_bias = np.log([cant_refa/cant_norefa])
    #weight = cant_norefa/cant_refa
    print('Peso de clase "0" (sin refactura): {:.2f}'.format(weight_for_0))
    print('Peso de clase "1" (con refactura): {:.2f}'.format(weight_for_1))
    print('Bias inicial: ',initial_bias)
    if only_pos==False:
        return weight_for_0, weight_for_1, initial_bias
    else:
        return weight_for_1

def curva_ROC(fpr, tpr, roc_auc):  
    # method I: plt
    plt.figure(figsize=(5, 5))
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.4f' % roc_auc, color='DarkSeaGreen')
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'--', color='RoyalBlue')
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend()
    #plt.savefig("ROC"  + ".jpg")
    plt.show()

#definimos funciona para mostrar los resultados
def mostrar_resultados(y_test, pred_y, umbral=None):
    for i in range(0, len(pred_y)):
        if pred_y[i] > umbral:
            pred_y[i] = 1
        else:
            pred_y[i] = 0
    conf_matrix = confusion_matrix(y_test, pred_y)
    plt.figure(figsize=(5, 5))
    sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d", cmap='Blues');
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    #plt.savefig("matriz_confusion"  + ".jpg")
    plt.show()
    #print (classification_report(y_test, pred_y))
    
def MOSTRAR_METRICAS(y_test, pred_y, umbral=None, beta=None):
    for i in range(0, len(pred_y)):
        if pred_y[i] > umbral:
            pred_y[i] = 1
        else:
            pred_y[i] = 0
    [TN, FP],[FN,TP] = confusion_matrix(y_test, pred_y)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1_score = (2*recall*precision) / (recall + precision)
    g_mean = np.sqrt((TP / (TP + FN)) * (TN / (TN + FP)))
    print('Recall: %0.4f' % recall)
    print('Precision: %0.4f' % precision)
    print('F1-score: %0.4f' % f1_score)
    print('G-mean: %0.4f' % g_mean)
    if beta is not None:
        f_beta = (((1 + beta)**2) * recall * precision) / (((beta**2) * precision) + recall)
        print('F-measure: %0.4f' % f_beta)
        return recall, precision, f1_score, g_mean, f_beta
    return recall, precision, f1_score, g_mean
        
def AUCM_PLOT(archivo):
    # Calculo de porcentajes
    porc_TD = numpy.arange(0, 1.01, 0.01)
    porc_TR = numpy.arange(0, 1.01, 0.01)
    j = 0
    maximo = 0
    limite_pasado = False
    for umbral in numpy.arange(0, 1.01, 0.01):
        predprob_y = joblib.load(archivo)
        for i in range(0, len(predprob_y)):
            if predprob_y[i] > umbral:
                predprob_y[i] = 1
            else:
                predprob_y[i] = 0
        [TN, FP],[FN,TP] = confusion_matrix(y_test, predprob_y)
        porc_TD[j] = (TP + FP) / (TP + FP + TN + FN)
        porc_TR[j] = TP / (TP + FN)
        if maximo < (porc_TR[j] - porc_TD[j]):
            maximo = umbral
            maximo_registros = TP + FP
        if ((TP+FP) < 3500) & (limite_pasado==False):
            limite_umbral = umbral
            limite_refactura = porc_TR[j]
            revisar = TP + FP
            limite_pasado = True
        j+=1
        #print('interacion de: ', umbral)
    umbral = numpy.arange(0, 1.01, 0.01)
    # Calculo de areas
    rocm_total = simps(porc_TD, umbral)
    rocm_refa = simps(porc_TR, umbral)
    AUCM = rocm_refa - rocm_total
    print('AUCM = %0.5f' % AUCM)
    #grafica
    plt.figure(figsize=(5, 5))
    plt.title('Receiver Operating Characteristic Modified (ROCM)')
    plt.plot(umbral, porc_TD, 'b', label = 'AUC Total = %0.4f' % rocm_total, color='indianred')
    plt.legend(loc = 'lower right')
    plt.plot(umbral, porc_TR, 'b', label = 'AUC Refacturas = %0.4f' % rocm_refa, color='DarkSeaGreen')
    plt.legend(loc = 'lower right')
    plt.xlim([1.01, -0.01])
    plt.ylim([-0.01, 1.01])
    plt.ylabel('% del Total')
    plt.xlabel('% probabilidad de refactura desde')
    plt.legend()
    #plt.savefig("ROCM" + ".jpg")
    plt.show()
    return limite_umbral, limite_refactura, revisar, maximo, maximo_registros, AUCM

def Carga_Datos(archivo=None, limpia=False, filtrado=False):
    if limpia==True:
        if len(archivo)>1:
            dtype = {'NRO_SUMINISTRO': 'str', 'COD_DIAMETRO': 'str', 'LECTURA_TERRENO': 'float32', 'CLAVE_TERRENO': 'str',
                     'LECTURA_ANT': 'float32', 'CONSUMO_BASE': 'float32', 'CONSUMO_PROM': 'float32', 'CATEGORIA': 'str',
                     'TIP_DOCUMENTO': 'str', 'COD_LECTOR': 'str', 'COD_OBS': 'str', 'COD_LOCALIDAD': 'str', 'RECORR1': 'str',
                     'RECORR2': 'str', 'TIE_REMAR': 'str', 'ID_RELACION': 'str', 'COD_EMPRESA': 'str', 
                     'CLAVE_TERRENO_MES_ANT': 'str', 'CONS_BASE_MES_ANT': 'float32', 'COD_OBS_MES_ANT': 'str',
                     'CLAVE_TERRENO_MISMO_MES_ANNO_ANT': 'str', 'CONS_BASE_MISMO_MES_ANNO_ANT': 'float32',
                     'COD_OBS_MISMO_MES_ANNO_ANT': 'str', 'ANNO': 'str', 'MES': 'str', 'ANALISTA': 'float32', 
                     'TIENE_REFA': 'float32'}
            data1 = pd.read_csv(archivo[0], delimiter=';', dtype=dtype, low_memory=False)
            data2 = pd.read_csv(archivo[1], delimiter=';', dtype=dtype, low_memory=False)
            Data = pd.concat([data1, data2], ignore_index=True)
            del data1
            del data2
            Data = Data.drop('Unnamed: 0', axis=1)
        else:
            dtype = {'NRO_SUMINISTRO': 'str', 'COD_DIAMETRO': 'str', 'LECTURA': 'float32', 'LECTURA_TERRENO': 'float32',
                     'CLAVE_TERRENO': 'str', 'LECTURA_ANT': 'float32', 'CONSUMO_BASE': 'float32', 'CONSUMO_PROM': 'float32',
                     'CATEGORIA': 'str', 'TIP_DOCUMENTO': 'str', 'COD_LECTOR': 'str', 'COD_OBS': 'str', 'COD_LOCALIDAD': 'str',
                     'RECORR1': 'str', 'RECORR2': 'str', 'TIE_REMAR': 'str', 'ID_RELACION': 'str', 'COD_EMPRESA': 'str',
                     'CLAVE_LECTURA': 'str', 'CLAVE_TERRENO_MES_ANT': 'str', 'CONS_BASE_MES_ANT': 'float32', 'COD_OBS_MES_ANT': 'str',
                     'CLAVE_TERRENO_MISMO_MES_ANNO_ANT': 'str', 'CONS_BASE_MISMO_MES_ANNO_ANT': 'float32',
                     'COD_OBS_MISMO_MES_ANNO_ANT': 'str', 'ANNO': 'str', 'MES': 'str', 'ANALISTA': 'float32', 'TIENE_REFA': 'float32'}
            Data = pd.read_csv(archivo[0], delimiter=';', dtype=dtype, low_memory=False)
            #Data = Data.drop('Unnamed: 0', axis=1)
        print('Se Cargo base de datos')
        return Data
    if len(archivo)>1:
        #Se agrega el tipo de variable de cada colunma para visualización
        dtype = {'NRO_SUMINISTRO': 'str', 'COD_DIAMETRO': 'str', 'LECTURA': 'float32', 'LECTURA_TERRENO': 'float32',
                 'CLAVE_TERRENO': 'str', 'LECTURA_ANT': 'float32', 'CONSUMO_BASE': 'float32', 'CONSUMO_PROM': 'float32',
                 'CATEGORIA': 'str', 'TIP_DOCUMENTO': 'str', 'COD_LECTOR': 'str', 'COD_OBS': 'str', 'COD_LOCALIDAD': 'str',
                 'RECORR1': 'str', 'RECORR2': 'str', 'TIE_REMAR': 'str', 'ID_RELACION': 'str', 'COD_EMPRESA': 'str',
                 'CLAVE_LECTURA': 'str', 'CLAVE_TERRENO_MES_ANT': 'str', 'CONS_BASE_MES_ANT': 'float32', 'COD_OBS_MES_ANT': 'str',
                 'CLAVE_TERRENO_MISMO_MES_ANNO_ANT': 'str', 'CONS_BASE_MISMO_MES_ANNO_ANT': 'float32', 
                 'COD_OBS_MISMO_MES_ANNO_ANT': 'str', 'ANNO': 'str', 'MES': 'str', 'TIENE_REFA': 'float32'}
        #se leen los datos
        data1 = pd.read_csv(archivo[0], delimiter=';', dtype=dtype, low_memory=False)
        data2 = pd.read_csv(archivo[1], delimiter=';', dtype=dtype, low_memory=False)
        #se encadenan todos los chunks en una tabla
        Data = pd.concat([data1, data2], ignore_index=True)
        del data1
        del data2
        data3 = pd.read_csv(archivo[2], delimiter=';', dtype=dtype, low_memory=False)
        Data = pd.concat([Data, data3], ignore_index=True)
        del data3
        Data = Data.drop([len(Data)-1],axis=0)
    else:
        dtype = {'NRO_SUMINISTRO': 'str', 'COD_DIAMETRO': 'str', 'LECTURA': 'float32', 'LECTURA_TERRENO': 'float32',
                 'CLAVE_TERRENO': 'str', 'LECTURA_ANT': 'float32', 'CONSUMO_BASE': 'float32', 'CONSUMO_PROM': 'float32',
                 'CATEGORIA': 'str', 'TIP_DOCUMENTO': 'str', 'COD_LECTOR': 'str', 'COD_OBS': 'str', 'COD_LOCALIDAD': 'str',
                 'RECORR1': 'str', 'RECORR2': 'str', 'TIE_REMAR': 'str', 'ID_RELACION': 'str', 'COD_EMPRESA': 'str',
                 'CLAVE_LECTURA': 'str', 'CLAVE_TERRENO_MES_ANT': 'str', 'CONS_BASE_MES_ANT': 'float32', 'COD_OBS_MES_ANT': 'str',
                 'CLAVE_TERRENO_MISMO_MES_ANNO_ANT': 'str', 'CONS_BASE_MISMO_MES_ANNO_ANT': 'float32',
                 'COD_OBS_MISMO_MES_ANNO_ANT': 'str', 'ANNO': 'str', 'MES': 'str', 'ANALISTA': 'float32', 'TIENE_REFA': 'float32'}
        Data = pd.read_csv(archivo[0], delimiter=';', dtype=dtype, low_memory=False)
    #Reemplazo de nulos -------------------------------------------------------------------------------------------------------
    Data['COD_DIAMETRO']=Data['COD_DIAMETRO'].fillna('13')
    Data['CATEGORIA']=Data['CATEGORIA'].fillna('R')
    Data['COD_LECTOR']=Data['COD_LECTOR'].fillna('0')
    Data['TIE_REMAR']=Data['TIE_REMAR'].fillna('N')
    Data['COD_OBS']=Data['COD_OBS'].fillna('ACE')
    Data['COD_OBS_MES_ANT']=Data['COD_OBS_MES_ANT'].fillna('ACE')
    Data['COD_OBS_MISMO_MES_ANNO_ANT']=Data['COD_OBS_MISMO_MES_ANNO_ANT'].fillna('ACE')
    Data['CLAVE_TERRENO_MES_ANT']=Data['CLAVE_TERRENO_MES_ANT'].fillna('N')
    Data['CLAVE_TERRENO_MISMO_MES_ANNO_ANT']=Data['CLAVE_TERRENO_MISMO_MES_ANNO_ANT'].fillna('N')
    Data['CONSUMO_BASE']=Data['CONSUMO_BASE'].fillna(0)
    Data['CONS_BASE_MES_ANT']=Data['CONS_BASE_MES_ANT'].fillna(0)
    #pre-procesamiento de algunas columnas ------------------------------------------------------------------------------------
    Data.CATEGORIA = Data.CATEGORIA.replace({"G": 'R', "1": 'R', "O": 'F'})
    Data.COD_DIAMETRO = Data.COD_DIAMETRO.replace({"13": '013'})
    Data = Data.drop(Data[Data['CATEGORIA']=='F'].index)
    #Filtrado de datos -----------------------------------------------------------------------------------------------------------
    if filtrado == True:
        #Filtrado casos atipicos 03-2016 LISTO
        Data = Data.drop(Data[((Data['MES']=='3') & (Data['ANNO']=='2016') & (Data['COD_EMPRESA']=='1') & (Data['TIENE_REFA']==1)) &
                              ((Data['COD_OBS']=='ACE') | (Data['COD_OBS']=='DXS') | (Data['COD_OBS']=='DHS') | (Data['COD_OBS']=='CXS') | (Data['COD_OBS']=='LAM') | (Data['COD_OBS']=='ASM') | (Data['COD_OBS']=='SUS')
                               | (Data['COD_OBS']=='BYP')) &
                              ((Data['CLAVE_TERRENO']=='1') | (Data['CLAVE_TERRENO']=='L') | (Data['CLAVE_TERRENO']=='N') | (Data['CLAVE_TERRENO']=='R'))
                             ].index)
        #Filtrado casos atipicos 10-2016 LISTO
        Data = Data.drop(Data[((Data['MES']=='10') & (Data['ANNO']=='2016') & (Data['COD_EMPRESA']=='2') & (Data['TIENE_REFA']==1)) &
                              ((Data['COD_OBS']=='ACE') | (Data['COD_OBS']=='CVE')) &
                              (Data['CLAVE_TERRENO']=='N')].index)
        #Filtrado casos atipicos 05-2017 LISTO
        Data = Data.drop(Data[((Data['MES']=='5') & (Data['ANNO']=='2017') & (Data['COD_EMPRESA']=='2') & (Data['TIENE_REFA']==1)) &
                              ((Data['RECORR1']=='08')|(Data['RECORR1']=='07')|(Data['RECORR1']=='06')|(Data['RECORR1']=='05')|
                               (Data['RECORR1']=='04')|(Data['RECORR1']=='03')|(Data['RECORR1']=='02')|(Data['RECORR1']=='01')) &
                              ((Data['COD_OBS']=='ACE') | (Data['COD_OBS']=='CC') | (Data['COD_OBS']=='DHS') | (Data['COD_OBS']=='CVE') | (Data['COD_OBS']=='LAM') | (Data['COD_OBS']=='ASM') | (Data['COD_OBS']=='SUS') | (Data['COD_OBS']=='CVA')) &
                              ((Data['CLAVE_TERRENO']=='N') | (Data['CLAVE_TERRENO']=='2') | (Data['CLAVE_TERRENO']=='1') | (Data['CLAVE_TERRENO']=='8') | (Data['CLAVE_TERRENO']=='I') | (Data['CLAVE_TERRENO']=='L') | (Data['CLAVE_TERRENO']=='P'))].index)
        #Filtrado casos atipicos 06-2017
        Data = Data.drop(Data[((Data['MES']=='6') & (Data['ANNO']=='2017') & (Data['COD_EMPRESA']=='2') & (Data['TIENE_REFA']==1)) &
                              ((Data['RECORR1']=='09')|(Data['RECORR1']=='10')|(Data['RECORR1']=='11')|(Data['RECORR1']=='12')|
                               (Data['RECORR1']=='13')|(Data['RECORR1']=='14')|(Data['RECORR1']=='15')|(Data['RECORR1']=='16')|
                               (Data['RECORR1']=='17')|(Data['RECORR1']=='18')|(Data['RECORR1']=='19')|(Data['RECORR1']=='20')) &
                              ((Data['COD_OBS']=='ACE') | (Data['COD_OBS']=='CC') | (Data['COD_OBS']=='DHS') | (Data['COD_OBS']=='CVE') | (Data['COD_OBS']=='LAM') | (Data['COD_OBS']=='ASM') | (Data['COD_OBS']=='SUS') | (Data['COD_OBS']=='DPU') | (Data['COD_OBS']=='LDE')) &
                              ((Data['CLAVE_TERRENO']=='N') | (Data['CLAVE_TERRENO']=='2') | (Data['CLAVE_TERRENO']=='1') | (Data['CLAVE_TERRENO']=='8') | (Data['CLAVE_TERRENO']=='3') | (Data['CLAVE_TERRENO']=='I') | (Data['CLAVE_TERRENO']=='L') | (Data['CLAVE_TERRENO']=='P') | (Data['CLAVE_TERRENO']=='T'))].index)
        #Filtrado casos atipicos 06-2018 LISTO
        Data = Data.drop(Data[((Data['MES']=='6') & (Data['ANNO']=='2018') & (Data['COD_EMPRESA']=='1') & (Data['TIENE_REFA']==1)) &
                              (Data['COD_OBS']=='CC') &
                              ((Data['CLAVE_TERRENO']=='1') | (Data['CLAVE_TERRENO']=='2'))].index)
    else:
        Data = Data.drop(Data[(Data['MES']=='3') & (Data['ANNO']=='2016')].index)
        Data = Data.drop(Data[(Data['MES']=='10') & (Data['ANNO']=='2016')].index)
        Data = Data.drop(Data[(Data['MES']=='5') & (Data['ANNO']=='2017')].index)
        Data = Data.drop(Data[(Data['MES']=='6') & (Data['ANNO']=='2017')].index)
        Data = Data.drop(Data[(Data['MES']=='6') & (Data['ANNO']=='2018')].index)
    #Reordenar los indices de dataframe
    Data = Data.reset_index(drop=True)
    #Agregar columna de responsabilidad del analista --------------------------------------------------------------------------
    Data.insert(len(Data.columns)-1, 'ANALISTA', 0)
    #codigos de responsabilidad del analista
    Data.loc[Data['CLAVE_LECTURA'] == 'P', 'ANALISTA'] = 1
    Data.loc[Data['CLAVE_LECTURA'] == 'A', 'ANALISTA'] = 1
    Data.loc[Data['CLAVE_LECTURA'] == 'B', 'ANALISTA'] = 1
    Data.loc[Data['CLAVE_LECTURA'] == 'C', 'ANALISTA'] = 1
    Data.loc[Data['CLAVE_LECTURA'] == 'G', 'ANALISTA'] = 1
    Data.loc[Data['CLAVE_LECTURA'] == 'M', 'ANALISTA'] = 1
    Data.loc[Data['CLAVE_LECTURA'] == 'O', 'ANALISTA'] = 1
    Data.loc[Data['CLAVE_LECTURA'] == 'W', 'ANALISTA'] = 1
    Data.loc[Data['CLAVE_LECTURA'] == 'Z', 'ANALISTA'] = 1
    
    print('Se Cargo base de datos')
    return Data

def Preprocesamiento_Datos(Data, ohe=None, mini=None, maxi=None, SS=None, SS_num=None, verbose=1):
    if (ohe is None) and (mini is None) and (maxi is None) and (SS is None) and (SS_num is None):
        codificacion=True
    else:
        codificacion=False
    
    #eliminacion de algunas columnas que no sirven
    Data = Data.drop('NRO_SUMINISTRO', axis=1)
    Data = Data.drop('COD_LECTOR', axis=1)
    Data = Data.drop('ANNO', axis=1)
    
    #guardar columnas que sean float
    columnas = Data.columns
    col = np.zeros(len(Data.columns))
    j = 0
    for i in columnas:
        if Data[i].dtype == 'f':
            col[j] = 1
        j = j + 1
            
    x_train = Data.values
    del Data
            
    x_train = x_train.astype(str)

    ######################################################################################################################
    #Preparar datos para codificacion
    x_train = np.transpose(x_train)
            
    #Separar datos numericos de categoricos
    init = 0
    k = 0
    for i in range(0, len(col)-1):
        if col[i] == 1:
            if init == 2:
                x_train_num = np.append(x_train_num, [x_train[k]], axis=0)
                x_train = np.delete(x_train, k, axis=0)
                k = k - 1
            if init == 1:
                x_train_num = np.append([x_train_num], [x_train[k]], axis=0)
                x_train = np.delete(x_train, k, axis=0)
                k = k - 1
                init = 2
            if init == 0:
                x_train_num = x_train[k]
                x_train = np.delete(x_train, k, axis=0)
                k = k - 1
                init = 1
            k = k + 1
        else:
            k = k + 1
    
    #codificacion OneHot para datos categorios ---------------------------------------------------------------------------
    x_train = np.transpose(x_train)
    
    if ohe is None:
        ohe = OneHotEncoder(dtype='float16',handle_unknown='ignore')
        ohe.fit(x_train)
    
    x_train = ohe.transform(x_train)
    
    #StandarScaler
    if SS is None:
        SS = StandardScaler(with_mean=False)
        SS.fit(x_train) 
    x_train = SS.transform(x_train)
    
    x_train = np.transpose(x_train)
            
    #estandarizacion minimax para datos numericos ----------------------------------------------------------------------
    x_train_num = x_train_num.astype('float')
    
    if (mini is None) & (maxi is None):
        mini = np.zeros(len(x_train_num))
        maxi = np.zeros(len(x_train_num))
        for i in range(0, len(x_train_num)):
            x_train_num[:,i], mini[i], maxi[i] = norm_minimax(datos_train=x_train_num[:,i], splits=False)
    if (mini is not None) & (maxi is not None):
        for i in range(len(x_train_num)):
            x_train_num[i,:] = norm_minimax(datos_train=x_train_num[i,:], splits=False, mini=mini[i], maxi=maxi[i])
    
    #StandarScaler  
    x_train_num = np.transpose(x_train_num)
    if SS_num is None:
        SS_num = StandardScaler()
        SS_num.fit(x_train_num)
    x_train_num = SS_num.fit_transform(x_train_num)
    
    x_train_num = np.transpose(x_train_num)
    
    #volver a unir los datos -------------------------------------------------------------------------------------------
    x_train = scipy.sparse.vstack((x_train, x_train_num))
    del x_train_num
    
    x_train = np.transpose(x_train)

    if verbose == 1:
        print('Codificacion lista')
        print('Dimensiones datos: ',x_train.shape)
    
    
    if (codificacion==False):
        return x_train
    
    else: 
        print('Error en argumentos de funcion de pre-procesamiento de datos')
        return -1
    #----------------------------------------------------------------------------------------------------------------------
    
#metricas ha ocupar
metricas = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
]
print('librerias y funciones listas')

Using TensorFlow backend.


librerias y funciones listas


In [2]:
start_time = datetime.now()
#Carga datos
archivo = ['../data-analista.csv']
Data = Carga_Datos(archivo=archivo, limpia=False)

# agregar columna de diferencia de consumos por temporada
data_consumos = joblib.load('../data_consumos.pkl')
Data = Data.merge(data_consumos, on='NRO_SUMINISTRO', how='left')
Data.insert(len(Data.columns)-2, 'DIFF_CONSUMOS', 0)
Data['DIFF_CONSUMOS'] = Data['diff_consumos']
Data = Data.drop('diff_consumos', axis=1)
Data = Data.dropna()

# eliminacion de algunas columnas que no sirven
Data = Data.drop('ANALISTA', axis=1)

print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Se Cargo base de datos
Time elapsed (hh:mm:ss.ms) 0:00:10.674681


In [3]:
from keras.models import model_from_json

start_time = datetime.now()

# cargar codificaciones
ohe = joblib.load('../ohe.pkl')
mini = joblib.load('../mini.pkl')
maxi = joblib.load('../maxi.pkl')
SS = joblib.load('../SS.pkl')
SS_num = joblib.load('../SS_num.pkl')
le = joblib.load('../le.pkl')

#definicion de temporadas
temporadas = ['1', '2', 'neutra']

predprob_y = np.array([])
y_test = np.array([])
for temporada in temporadas:
    if temporada == '1':
        Data_test_temp = Data.drop(Data[Data['DIFF_CONSUMOS'] <= 0.5].index)
    if temporada == '2':
        Data_test_temp = Data.drop(Data[Data['DIFF_CONSUMOS'] >= -0.5].index)
    if temporada == 'neutra':
        Data_test_temp = Data.drop(Data[(Data['DIFF_CONSUMOS'] > 0.5) | (Data['DIFF_CONSUMOS'] < -0.5)].index)
            
    Data_test_temp = Data_test_temp.drop('DIFF_CONSUMOS', axis=1)
    
    if len(Data_test_temp) != 0:
        # guardar numero de suministro de clientes
        if temporada == '1':
            registros = Data_test_temp['NRO_SUMINISTRO']
        else:
            registros = pd.concat([registros, Data_test_temp['NRO_SUMINISTRO']], axis=0)
        # codificacion
        x_test = Preprocesamiento_Datos(Data_test_temp,
                                        ohe=ohe, mini=mini, maxi=maxi, SS=SS, SS_num=SS_num, verbose=0)
        # cargando el modelo desde el disco
        filename = 'archivos_carga_modelos/modelo_NN_DA_temporada_' + str(temporada) + '.json'
        filename_pesos = 'archivos_carga_modelos/pesos_NN_DA_temporada_' + str(temporada) + '.h5'
        json_file = open(filename, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model_NN = model_from_json(loaded_model_json)
        # cargar los pesos del nuevo modelo
        model_NN.load_weights(filename_pesos)
        # compilar el modelo
        model_NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=metricas)
        # hacer prediccion
        predprob_y = np.append(predprob_y, model_NN.predict_proba(x_test.tocsr()))
        #y_test = np.append(y_test, y_test_temp)

# resetiar indices de numero de registro
registros = registros.reset_index(drop=True)
        
# pasar las probabilidades de predicion a dataframe
predicciones = pd.DataFrame(predprob_y, columns = ['PROB_REFACTURA'])

#unir ambos dataframe
salida = pd.concat([registros, predicciones], axis=1)

# crear archivo de salida
nombre_salida = 'prob_refactura_NN_DA_temporada.csv' 
salida.to_csv(nombre_salida, sep=';', index=False)
print('Se guardo el archivo de salida')

print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Se guardo el archivo de salida
Time elapsed (hh:mm:ss.ms) 0:00:04.458329
